In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation , Add , Input
from scipy.signal.windows import gaussian
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from keras.callbacks import ModelCheckpoint
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))

In [2]:
img_height = 224
img_width = 224
batch_size = 32

In [3]:
img_height = 224
img_width = 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)

base_dirs = {
    'train': r'datasets/real_combined_last_dataset/train',
    'test': r'datasets/real_combined_last_dataset/test'
}

In [4]:
def data_generator(directory):
    return datagen.flow_from_directory(
        directory,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary',  # Update to 'categorical' for classification or 'input' for segmentation
        shuffle=False,  # Set to True if you want to shuffle the data
        seed=42
    )

train_ds = data_generator(base_dirs['train'])
test_ds = data_generator(base_dirs['test'])

Found 6359 images belonging to 3 classes.
Found 818 images belonging to 3 classes.


In [5]:
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)

    # Apply Gaussian smoothing
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Split the image into B,G,R channels
    B, G, R = cv2.split(image)

    # Apply histogram equalization to each channel
    B_equ = cv2.equalizeHist(B)
    G_equ = cv2.equalizeHist(G)
    R_equ = cv2.equalizeHist(R)

    # Merge the channels back together
    equ = cv2.merge((B_equ, G_equ, R_equ))

    # Resize the image
    equ = cv2.resize(equ, target_size)

    # Normalize pixel values
    equ = equ / 255.0

    return equ


In [6]:
def segment(image, plot_seg, plot_hist):
    output_directory = 'datasets/real_combined_last_dataset/Cups_discs1'
    # Pre-processing and Smoothing
    image = image.astype('uint8')
    Abo, Ago, Aro = cv2.split(image)
    Aro = clahe.apply(Aro)
    Ago = Ago.astype('uint8')
    Ago = clahe.apply(Ago)
    # Gaussian Window for smoothing
    M = 60
    filter = gaussian(M, std=6)
    filter = filter / sum(filter)
    STDf = filter.std()

    # Preprocessing Red and Green Channels
    Ar = Aro - Aro.mean() - Aro.std()
    Mr = Ar.mean()
    SDr = Ar.std()
    #print("Mean Ar:", Ar.mean())
    #print("Std Dev Ar:", Ar.std())
    Thr = Ar.mean() - 0.5 * Ar.std()  # Adjust as needed
    #Thr = 0.5 * M - STDf - Ar.std()
    #print("THR",Thr)
    Ag = Ago - Ago.mean() - Ago.std()
    Mg = Ag.mean()
    SDg = Ag.std()
    Thg = 0.5 * Mg + 2 * STDf + 2 * SDg + Mg
    #print("Thg",Thg)
    # Histogram Calculation
    hist, bins = np.histogram(Ag.ravel(), 256, [0, 256])
    histr, binsr = np.histogram(Ar.ravel(), 256, [0, 256])

    # Histogram Smoothing
    smooth_hist_g = np.convolve(filter, hist)
    smooth_hist_r = np.convolve(filter, histr)

    if plot_hist:
        plt.subplot(2, 2, 1)
        plt.plot(hist)
        plt.title("Preprocessed Green Channel")
        plt.subplot(2, 2, 2)
        plt.plot(smooth_hist_g)
        plt.title("Smoothed Histogram Green Channel")
        plt.subplot(2, 2, 3)
        plt.plot(histr)
        plt.title("Preprocessed Red Channel")
        plt.subplot(2, 2, 4)
        plt.plot(smooth_hist_r)
        plt.title("Smoothed Histogram Red Channel")
        plt.show()
    Ar = Ar.astype('uint8')
    Ag = Ag.astype('uint8')
    # Binary decision maps for Optic Disk and Optic Cup
    _, Dd = cv2.threshold(Ar, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, Dc = cv2.threshold(Ag, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    os.makedirs(output_directory, exist_ok=True)
    cv2.imwrite(os.path.join(output_directory, 'disk.png'), Dd)
    plt.imsave(os.path.join(output_directory, 'cup.png'), Dc)

    if plot_seg:
        plt.subplot(2, 2, 1)
        plt.imshow(Dd, cmap='gray')
        plt.axis("off")
        plt.title("Optic Disk")
        plt.subplot(2, 2, 2)
        plt.imshow(Dc, cmap='gray')
        plt.axis("off")
        plt.title("Optic Cup")
        plt.show()


    return Dd, Dc

In [7]:
def cdr(cup, disc, plot):
    # Convert cup and disc images to uint8 type
    cup = cup.astype('uint8')
    disc = disc.astype('uint8')

    # Morphological operations on cup
    R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    img = clahe.apply(r3)

    # Find contours in cup
    ret, thresh = cv2.threshold(cup, 127, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cup_diameter = 0

    if contours:
        largest_area = 0
        el_cup = contours[0]
        for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i])
                if area > largest_area:
                    largest_area = area
                    el_cup = cv2.fitEllipse(contours[i])

        cv2.ellipse(img, el_cup, (140, 60, 150), 3)
        x, y, w, h = cv2.boundingRect(contours[0])
        cup_diameter = max(w, h)

    # Morphological operations on disc
    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)

    img2 = clahe.apply(r3)

    # Find contours in disc
    ret, thresh = cv2.threshold(disc, 127, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    disk_diameter = 0

    if contours:
        largest_area = 0
        el_disc = contours[0]
        for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i])
                if area > largest_area:
                    largest_area = area
                    el_disc = cv2.fitEllipse(contours[i])

        cv2.ellipse(img2, el_disc, (140, 60, 150), 3)
        x, y, w, h = cv2.boundingRect(contours[0])
        disk_diameter = max(w, h)


    if plot:
        plt.subplot(2,2,1)
        plt.imshow(img2, interpolation='bicubic')
        plt.axis("off")
        plt.title("Optic Disk ")
        plt.subplot(2,2,2)
        plt.imshow(img)
        plt.axis("off")
        plt.title("Optic Cup ")
        plt.show()

    if disk_diameter == 0:
        return 1

    cdr = cup_diameter / disk_diameter #cup_diameter / disk_diameter
    return cdr


In [8]:
def process_image(image_path, model):
    image = preprocess_image(image_path)
    cup, disc = segment(image, plot_seg = False, plot_hist = False )
    cdr_value = cdr(cup, disc, plot=False)
    return cdr_value

In [9]:
def unet_model(train_ds, test_ds):
    model = Sequential()

    # Encoder
    model.add(Conv2D(64, (3, 3), input_shape=(img_height, img_width, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Decoder
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(UpSampling2D(size=(2, 2)))

    # Output layer
    model.add(Conv2D(1, (1, 1), activation='sigmoid', padding='same'))
    checkpoint_path = 'datasets/real_combined_last_dataset/model_checkpoint.h5'

    if os.path.exists(checkpoint_path):
        print(f"Loading weights from checkpoint: {checkpoint_path}")
        try:
            model.load_weights(checkpoint_path)
            print("Weights loaded successfully.")
        except Exception as e:
            print(f"Error loading weights: {str(e)}")
    else:
        print(f"Checkpoint file not found at {checkpoint_path}")
    
    model_checkpoint = ModelCheckpoint(checkpoint_path, save_weights_only=True, save_best_only=False, save_freq='epoch')
    print("Training model")
    optimizer = Adam(learning_rate=1e-5)  # Use the updated optimizer
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])
    classifier = model.fit(train_ds, epochs=10, callbacks=[model_checkpoint])
    model_saver = model.save('datasets/real_combined_last_dataset/unet_model.h5')
    test_result = model.evaluate(test_ds)
    print(f"Test Loss: {test_result[0]}, Test Accuracy: {test_result[1]}")

    predictions = model.predict(test_ds)
    predictions_flat = predictions.reshape(predictions.shape[0], -1)
    column_names = [f"Pixel_{i}" for i in range(predictions_flat.shape[1])]
    df = pd.DataFrame(predictions_flat, columns=column_names)
    df.to_csv('datasets/real_combined_last_dataset/cup_and_disc_segmentation_results(copy of unet running again1).csv', index=False)

    # Save the entire model
    #model_saver = model.save('datasets/real_combined_last_dataset/unet_model.h5')
    return model

In [10]:
def process(base_dirs):
    all_data = []
    categories = ['glaucoma', 'normal', 'early_glaucoma']

    model = unet_model(train_ds,test_ds)

    #checkpoint_path = 'datasets/real_combined_last_dataset/model_checkpoint(copy of unet running again1).h5'
    #initial_epoch = get_last_epoch(model, checkpoint_path)

    # Train for 10 epochs initially
    #train_model(model, train_ds, test_ds)
    #train_model(model, train_ds, test_ds, initial_epoch=initial_epoch + 1, extra_epochs=1)

    for dataset_type, base_dir in base_dirs.items():
        for category in categories:
            category_dir = os.path.join(base_dir, category)
            for filename in os.listdir(category_dir):
                try:
                    if filename.endswith(".jpg") or filename.endswith(".png"):
                        image_path = os.path.join(category_dir, filename)

                        cdr_value = process_image(image_path, model)

                        if category == 'glaucoma':
                            label = 1
                        elif category == 'early_glaucoma':
                            label = -1
                        elif category == 'normal':
                            label = 0

                        all_data.append({
                            'filename': str(filename),
                            'label': label,
                            'cdr': cdr_value
                        })
                        print(f"{filename}, label: {label} - Pred_cdr: {cdr_value}")
                except Exception as e:
                    print(f"Error processing image: {image_path}")
                    print(f"Error message: {str(e)}")
                    break

    df = pd.DataFrame(all_data)
    output_directory = "/datasets/real_combined_last_dataset"
    os.makedirs(output_directory, exist_ok=True)
    output_file_path = os.path.join(output_directory, 'cupanddisc(cdr)(copy of unet running again1).csv')
    df.to_csv(output_file_path, index=False)

In [14]:
process = process(base_dirs)

Loading weights from checkpoint: datasets/real_combined_last_dataset/model_checkpoint.h5
Weights loaded successfully.
Training model
Epoch 1/10
199/199 [==============================] - 2716s 14s/step - loss: 0.3230 - accuracy: 0.7074
Epoch 2/10
199/199 [==============================] - 2123s 11s/step - loss: 0.3202 - accuracy: 0.7082
Epoch 3/10
199/199 [==============================] - 2020s 10s/step - loss: 0.3180 - accuracy: 0.7088
Epoch 4/10
199/199 [==============================] - 2160s 11s/step - loss: 0.3161 - accuracy: 0.7094
Epoch 5/10
199/199 [==============================] - 2160s 11s/step - loss: 0.3143 - accuracy: 0.7098
Epoch 6/10
199/199 [==============================] - 2265s 11s/step - loss: 0.3123 - accuracy: 0.7105
Epoch 7/10
199/199 [==============================] - 2407s 12s/step - loss: 0.3110 - accuracy: 0.7110
Epoch 8/10
199/199 [==============================] - 2500s 13s/step - loss: 0.3100 - accuracy: 0.7113
Epoch 9/10
199/199 [=======================

E:\study material\miniproject-2\GalacticGrove\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - 68s 3s/step - loss: 0.3254 - accuracy: 0.6930
Test Loss: 0.3254264295101166, Test Accuracy: 0.6929606795310974
26/26 [==============================] - 61s 2s/step
069.jpg, label: 1 - Pred_cdr: 0.1875
072.jpg, label: 1 - Pred_cdr: 0.045454545454545456
086.jpg, label: 1 - Pred_cdr: 0.058823529411764705
087.jpg, label: 1 - Pred_cdr: 0.10526315789473684
094.jpg, label: 1 - Pred_cdr: 1.0
097.jpg, label: 1 - Pred_cdr: 0.1875
1.png, label: 1 - Pred_cdr: 0.3076923076923077
10.png, label: 1 - Pred_cdr: 0.1111111111111111
100.png, label: 1 - Pred_cdr: 0.1875
101.png, label: 1 - Pred_cdr: 3.3333333333333335
102.png, label: 1 - Pred_cdr: 0.375
103.png, label: 1 - Pred_cdr: 0.2777777777777778
104.png, label: 1 - Pred_cdr: 0.42857142857142855
106.png, label: 1 - Pred_cdr: 0.6666666666666666
108.png, label: 1 - Pred_cdr: 0.1111111111111111
109.png, label: 1 - Pred_cdr: 0.21428571428571427
11.png, label: 1 - Pred_cdr: 0.06666666666666667
110.jpg, label: 1 - Pr